In [1]:
import tifffile 
import numpy as np
from bigstream.align import alignment_pipeline
from bigstream.transform import apply_transform
import napari

import nd2 # reading nd2 file
import dask # allows reading data in blocks, used for reading large nd2 files

In [2]:
spacing = np.array([1, .17, .17])
C = [0,1,2,3] 
# ZCYX.tiff

affine_kwargs = {
    'alignment_spacing':2.0,
    'shrink_factors':(2,),
    'smooth_sigmas':(2.,),
    'optimizer_args':{
        'learningRate':0.25,
        'minStep':0.,
        'numberOfIterations':800, 
    },
}

steps = [('affine', affine_kwargs,),] # an option here is to deform: ('deform', deform_kwargs,) see deformable_align function in align.py and configure_irm.py for parameters

In [5]:
for n in range(0,42):     # loop for all fields of view
    print(n)
    fix_path = '.\images\Time00002_Channel640,561,488,405_Seq0002.nd2'
    fix = nd2.imread(fix_path,dask=True)[n,:,3,:,:].compute() # read DAPI channel, the 4th channel
    
    for x in range(0,7):    # loop for all cycles, each cycle as a 4-channel image
        print(x)
        mov_path = '.\images\Time0000'+ str(x) + '_Channel640,561,488,405_Seq000' + str(x) + '.nd2'
        mov = nd2.imread(mov_path,dask=True)[n,:,3,:,:].compute()
        print(fix.shape,mov.shape)
        
        affine = alignment_pipeline(
        fix, mov, # fix data first
        spacing, spacing, # fix spacing first, but the spacing is the same in this case
        steps=steps
    )

    # np.savetxt('Affine'+str(x)+'.mat', affine)
    # # load result
    # affine = np.loadtxt('Affine'+str(x)+'.mat')

        C = [0,1,2,3]
        for c in C:     # apply transformation for all channels
            print(c)
            # mov2 = tifffile.imread(mov_path)[:,c,:,:].squeeze()      # for tiff files
            mov2 = nd2.imread(mov_path,dask=True)[n,:,c,:,:].compute()      # for nd2 files
            
            aligned = apply_transform(
                fix, mov2, spacing, spacing,
                transform_list=[affine,], # 'affine' input here is the transformation matrix we got based on DAPI data
            )
            
            out_path = './Bigstream_notebook/Aligned_zstack'+ str(n) + '_Time'+str(x)+ '_channel'+str(c)+'.tiff'
            print(out_path)
            tifffile.imwrite(out_path, aligned, imagej=True, metadata={'axes':'ZYX'})


0


C:\Users\liany\AppData\Local\anaconda3\Lib\site-packages\dask\base.py:1252: UserWarning: ND2File file not closed before garbage collection. Please use `with ND2File(...):` context or call `.close()`.
  mod.loads(out, buffers=buffers)


3


C:\Users\liany\AppData\Local\anaconda3\Lib\site-packages\dask\base.py:1252: UserWarning: ND2File file not closed before garbage collection. Please use `with ND2File(...):` context or call `.close()`.
  mod.loads(out, buffers=buffers)


(61, 2304, 2304) (61, 2304, 2304)
LEVEL:  0  ITERATION:  0  METRIC:  -0.23901837399718337
LEVEL:  0  ITERATION:  1  METRIC:  -0.2582960109465284
LEVEL:  0  ITERATION:  2  METRIC:  -0.25876369198303434
LEVEL:  0  ITERATION:  3  METRIC:  -0.26184138675098956
LEVEL:  0  ITERATION:  4  METRIC:  -0.2619457023058708
LEVEL:  0  ITERATION:  5  METRIC:  -0.26210868323433845
LEVEL:  0  ITERATION:  6  METRIC:  -0.2621823691909654
LEVEL:  0  ITERATION:  7  METRIC:  -0.2622343051337359
LEVEL:  0  ITERATION:  8  METRIC:  -0.262314058543613
LEVEL:  0  ITERATION:  9  METRIC:  -0.26235715028917916
LEVEL:  0  ITERATION:  10  METRIC:  -0.2624252788063606
LEVEL:  0  ITERATION:  11  METRIC:  -0.26247098507619065
LEVEL:  0  ITERATION:  12  METRIC:  -0.2625352731257136
LEVEL:  0  ITERATION:  13  METRIC:  -0.26256728495885995
LEVEL:  0  ITERATION:  14  METRIC:  -0.2624368239803192
LEVEL:  0  ITERATION:  15  METRIC:  -0.2626269478123956
LEVEL:  0  ITERATION:  16  METRIC:  -0.2627053390874489
LEVEL:  0  ITERATI

...

./Bigstream_notebook/Aligned_zstack40_Time6_channel2.tiff
3


C:\Users\liany\AppData\Local\anaconda3\Lib\site-packages\dask\base.py:1252: UserWarning: ND2File file not closed before garbage collection. Please use `with ND2File(...):` context or call `.close()`.
  mod.loads(out, buffers=buffers)


./Bigstream_notebook/Aligned_zstack40_Time6_channel3.tiff
41


C:\Users\liany\AppData\Local\anaconda3\Lib\site-packages\dask\base.py:1252: UserWarning: ND2File file not closed before garbage collection. Please use `with ND2File(...):` context or call `.close()`.
  mod.loads(out, buffers=buffers)


IndexError: Index 41 is out of bounds for axis 0 with size 41

In [29]:
# use napari to look at the two DAPI data
# use subsampling of 8,4,4, the images have lots of data, but we don't actually need to see all of the data points

fix_img = fix[::8,::4,::4]
mov_img = mov[::8,::4,::4]

viewer = napari.Viewer()
viewer.add_image(fix_img,colormap='reds',opacity=0.4, blending='additive')
viewer.add_image(mov_img,colormap='blues',opacity=0.4, blending='additive')

# swtich between the channels, or use different colors and decrease opacity to check if they are exactly aligned

<Image layer 'mov_img' at 0x22119d47cd0>

In [7]:
### we can also use napari to compare 3 data: fix, transformed mov, untransformed mov
aligned_img = aligned[::8,::4,::4]

viewer = napari.Viewer()
viewer.add_image(fix_img,colormap='reds', opacity=0.4, blending='additive')
viewer.add_image(mov_img,colormap='blues', opacity=0.4, blending='additive')
viewer.add_image(aligned_img,colormap='greens', opacity=0.4, blending='additive')

# aligned data should look more like the fix data

<Image layer 'aligned_img' at 0x2305ae8bed0>